# Importing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee
import geemap
import fiona
import time
import json
from pathlib import Path
import datetime
import numpy as np
from utils.gee import export_sar, export_opt, export_maskcloud, export_opt_l1
import yaml

# Authentication

In [3]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


# Site Selection

In [90]:
site = 2
stie_cfg = f'site_{site}.yaml'

In [91]:
with open(stie_cfg, 'r') as file:
    cfg = yaml.safe_load(file)

In [92]:
cfg

{'xcenter': -55.184,
 'ycenter': -7.248,
 'xmin': -55.61317210385657,
 'ymin': -8.059505066063522,
 'xmax': -54.773337144733226,
 'ymax': -6.452868180415758,
 'date': ['2019-07-24', '2020-07-26', '2021-07-29'],
 'n_sar_avg_images': 12,
 'before_days': 31,
 'opt_images_dates': [['2019-07-16', '2019-07-17'],
  ['2019-07-31', '2019-08-01'],
  ['2019-08-05', '2019-08-06'],
  ['2020-07-30', '2020-07-31'],
  ['2020-08-19', '2020-08-20'],
  ['2020-06-30', '2020-07-01'],
  ['2021-07-20', '2021-07-21'],
  ['2021-07-30', '2021-07-31'],
  ['2021-08-14', '2021-08-15']],
 'tiles_params': {'lines': 4,
  'columns': 4,
  'train_tiles': [1, 2, 4, 5, 7, 8, 9, 10, 13, 15]},
 'original_data': {'opt_imgs': {'train': ['2019-07-16-2019-07-17.tif',
    '2019-07-31-2019-08-01.tif',
    '2019-08-05-2019-08-06.tif',
    '2020-07-30-2020-07-31.tif',
    '2020-08-19-2020-08-20.tif',
    '2020-06-30-2020-07-01.tif'],
   'test': ['2020-07-30-2020-07-31.tif',
    '2020-08-19-2020-08-20.tif',
    '2020-06-30-2020-07-0

# Reading Location data

In [93]:
roi = ee.Geometry.Rectangle([cfg['xmin'], cfg['ymin'], cfg['xmax'], cfg['ymax']])

In [ ]:
map_view = geemap.Map(center=(cfg['ycenter'], cfg['xcenter']), zoom=8)
map_view


In [95]:
#location = geemap.shp_to_ee(str(location_shape))
map_view.addLayer(roi, {}, 'Location')

# Dates

In [96]:
before_days = cfg['before_days']
d0 = datetime.datetime.strptime(cfg['date'][0], "%Y-%m-%d")
d1 = datetime.datetime.strptime(cfg['date'][1], "%Y-%m-%d")
d2 = datetime.datetime.strptime(cfg['date'][2], "%Y-%m-%d")
dt1 = (d1-d0).days + before_days
dt2 = (d2-d1).days + before_days

In [97]:
start_date = d0 - datetime.timedelta(days=before_days)
end_date = d2 + datetime.timedelta(days=before_days)
print(f'Analyzed Period: {start_date.date()}:{end_date.date()}')


Analyzed Period: 2019-06-23:2021-08-29


In [98]:
opt_vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 2048
}

sar_vizParams = {
  'bands': ['VV'],
  'min': 0,
  'max': 1
}

cloud_vizParams = {
  'bands': ['MSK_CLDPRB'],
  'min': 0,
  'max': 100
}

# SAR Data

## Average Data

In [68]:
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(str(start_date.date()), str(end_date.date())).filterBounds(roi)
ds = np.arange(dt1)
img_collections = []
split_ds = np.array_split(ds, cfg['n_sar_avg_images'])
for i, dsi in enumerate(split_ds):
    di0 = d0 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[0]))
    di1 = d0 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[-1]))
    img_col = sar_collection.filterDate(str(di0.date()), str(di1.date()))
    img = img_col.mean().clip(roi)
    #print(f'{di0.date()}-{di1.date()}-delta:{(di1-di0).days}|{len(img_col.getInfo()["features"])} images')
    print(f'{di0.date()}-{di1.date()}.tif')
    map_view.addLayer(img, sar_vizParams, f't1-{i:02d}')
    #export_sar(img, roi, f'{di0.date()}-{di1.date()}')
    

2019-06-23-2019-07-26.tif
2019-07-27-2019-08-29.tif
2019-08-30-2019-10-02.tif
2019-10-03-2019-11-04.tif
2019-11-05-2019-12-07.tif
2019-12-08-2020-01-09.tif
2020-01-10-2020-02-11.tif
2020-02-12-2020-03-15.tif
2020-03-16-2020-04-17.tif
2020-04-18-2020-05-20.tif
2020-05-21-2020-06-22.tif
2020-06-23-2020-07-25.tif


In [69]:
ds = np.arange(dt2)
img_collections = []
split_ds = np.array_split(ds,  cfg['n_sar_avg_images'])
for i, dsi in enumerate(split_ds):
    di0 = d1 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[0]))
    di1 = d1 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[-1]))
    img_col = sar_collection.filterDate(str(di0.date()), str(di1.date()))
    img = img_col.mean().clip(roi)
    #print(f'{di0.date()}-{di1.date()}-delta:{(di1-di0).days}|{len(img_col.getInfo()["features"])} images')
    print(f'{di0.date()}-{di1.date()}.tif')
    map_view.addLayer(img, sar_vizParams, f't2-{i:02d}')
    #export_sar(img, roi, f'{di0.date()}-{di1.date()}')

2020-06-25-2020-07-28
2020-07-29-2020-08-31
2020-09-01-2020-10-04
2020-10-05-2020-11-06
2020-11-07-2020-12-09
2020-12-10-2021-01-11
2021-01-12-2021-02-13
2021-02-14-2021-03-18
2021-03-19-2021-04-20
2021-04-21-2021-05-23
2021-05-24-2021-06-25
2021-06-26-2021-07-28


## Multiple Images

In [107]:
date_0, date_1 = d0 - datetime.timedelta(days=12), d1 + datetime.timedelta(days=12)
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(str(date_0.date()), str(date_1.date())).filterBounds(roi)
img_col = sar_collection.filter(ee.Filter.eq('platform_number', 'A'))
imgs = img_col.getInfo()['features']
print(len(imgs))
print(f'{date_0.date()}---{date_1.date()}')
cycles = set({})
for im in imgs:
    cycle = im['properties']['cycleNumber'] 
    cycles.add(cycle)
cycles = list(cycles)
print(len(cycles))
skip_idx = []
count = 0
for idx, cycle in enumerate(cycles):
    if idx in skip_idx:
        continue
    cycle_col = sar_collection.filter(ee.Filter.eq('cycleNumber', cycle))
    img = cycle_col.mosaic()
    data_check = img.mask().reduceRegion(
        ee.Reducer.min(), 
        roi,
        scale = 10,
        maxPixels = 1e9).getInfo()
    #if not 0 in data_check.values():
    if data_check['angle'] == 1:
        count+=1
        img = img.clip(roi)
        map_view.addLayer(img, sar_vizParams, f't2-{str(cycle)}')
        dates = set({})
        cycle_imgs = cycle_col.getInfo()["features"]
        for cycle_img in cycle_imgs:
            day = datetime.datetime.strptime(cycle_img['properties']['system:index'][17:25], '%Y%m%d')
            dates.add(day)
        dates = list(dates)
        dates.sort()
        print(f'{idx}: {dates[0].date()}-{dates[-1].date()}.tif')
        #export_sar(img, roi, f'{dates[0].date()}-{dates[-1].date()}')
print(count)

130
2019-07-12---2020-08-07
33
0: 2019-07-14-2019-07-21.tif
1: 2019-07-26-2019-08-02.tif
2: 2019-08-07-2019-08-14.tif
3: 2019-08-19-2019-08-26.tif
4: 2019-08-31-2019-09-07.tif
5: 2019-09-12-2019-09-19.tif
6: 2019-09-24-2019-10-01.tif
7: 2019-10-06-2019-10-13.tif
8: 2019-10-18-2019-10-25.tif
9: 2019-10-30-2019-11-06.tif
10: 2019-11-11-2019-11-18.tif
11: 2019-11-23-2019-11-30.tif
12: 2019-12-05-2019-12-12.tif
13: 2019-12-17-2019-12-24.tif
14: 2019-12-29-2020-01-05.tif
15: 2020-01-10-2020-01-17.tif
16: 2020-01-22-2020-01-29.tif
17: 2020-02-03-2020-02-10.tif
18: 2020-02-15-2020-02-22.tif
19: 2020-02-27-2020-03-05.tif
20: 2020-03-10-2020-03-17.tif
21: 2020-03-22-2020-03-29.tif
22: 2020-04-03-2020-04-10.tif
23: 2020-04-15-2020-04-22.tif
24: 2020-04-27-2020-05-04.tif
25: 2020-05-09-2020-05-16.tif
26: 2020-05-21-2020-05-28.tif
27: 2020-06-02-2020-06-09.tif
28: 2020-06-14-2020-06-21.tif
29: 2020-06-26-2020-07-03.tif
30: 2020-07-08-2020-07-15.tif
31: 2020-07-20-2020-07-27.tif
32


In [108]:
date_0, date_1 = d1 - datetime.timedelta(days=12), d2 + datetime.timedelta(days=12)
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(str(date_0.date()), str(date_1.date())).filterBounds(roi)
img_col = sar_collection.filter(ee.Filter.eq('platform_number', 'A'))
imgs = img_col.getInfo()['features']
print(len(imgs))
print(f'{date_0.date()}---{date_1.date()}')
cycles = set({})
for im in imgs:
    cycle = im['properties']['cycleNumber'] 
    cycles.add(cycle)
cycles = list(cycles)
print(len(cycles))
skip_idx = []
count = 0
for idx, cycle in enumerate(cycles):
    if idx in skip_idx:
        continue
    cycle_col = sar_collection.filter(ee.Filter.eq('cycleNumber', cycle))
    img = cycle_col.mosaic()
    data_check = img.mask().reduceRegion(
        ee.Reducer.min(), 
        roi,
        scale = 10,
        maxPixels = 1e9).getInfo()
    #if not 0 in data_check.values():
    if data_check['angle'] == 1:
        count+=1
        img = img.clip(roi)
        #map_view.addLayer(img, sar_vizParams, f't2-{str(cycle)}')
        dates = set({})
        cycle_imgs = cycle_col.getInfo()["features"]
        for cycle_img in cycle_imgs:
            day = datetime.datetime.strptime(cycle_img['properties']['system:index'][17:25], '%Y%m%d')
            dates.add(day)
        dates = list(dates)
        dates.sort()
        print(f'{idx}: {dates[0].date()}-{dates[-1].date()}.tif')
        export_sar(img, roi, f'{dates[0].date()}-{dates[-1].date()}')
print(count)

132
2020-07-14---2021-08-10
34
1: 2020-07-20-2020-07-27.tif
2: 2020-08-01-2020-08-08.tif
3: 2020-08-13-2020-08-20.tif
4: 2020-08-25-2020-09-01.tif
5: 2020-09-06-2020-09-13.tif
6: 2020-09-18-2020-09-25.tif
7: 2020-09-30-2020-10-07.tif
8: 2020-10-12-2020-10-19.tif
9: 2020-10-24-2020-10-31.tif
10: 2020-11-05-2020-11-12.tif
11: 2020-11-17-2020-11-24.tif
12: 2020-11-29-2020-12-06.tif
13: 2020-12-11-2020-12-18.tif
14: 2020-12-23-2020-12-30.tif
15: 2021-01-04-2021-01-11.tif
16: 2021-01-16-2021-01-23.tif
17: 2021-01-28-2021-02-04.tif
18: 2021-02-09-2021-02-16.tif
19: 2021-02-21-2021-02-28.tif
20: 2021-03-05-2021-03-12.tif
21: 2021-03-17-2021-03-24.tif
22: 2021-03-29-2021-04-05.tif
23: 2021-04-10-2021-04-17.tif
24: 2021-04-22-2021-04-29.tif
25: 2021-05-04-2021-05-11.tif
26: 2021-05-16-2021-05-23.tif
27: 2021-05-28-2021-06-04.tif
28: 2021-06-09-2021-06-16.tif
29: 2021-06-21-2021-06-28.tif
30: 2021-07-03-2021-07-10.tif
31: 2021-07-15-2021-07-22.tif
32: 2021-07-27-2021-08-03.tif
32


# Optical Data (Level 2A)

In [13]:
opt_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(start_date, end_date)
opt_collection = opt_collection.filterBounds(roi)
opt_info = opt_collection.getInfo()

In [28]:
for info in opt_info['features']:
    img_date = datetime.datetime.strptime(info['properties']['DATATAKE_IDENTIFIER'][5:13], '%Y%m%d')
    img_lct = info['properties']['PRODUCT_ID']
    cloud = info['properties']['CLOUDY_PIXEL_PERCENTAGE']
    print(img_date, img_lct, cloud)
    #break

2019-07-17 00:00:00 S2B_MSIL2A_20190717T142049_N0213_R010_T20MQS_20190717T193145 13.581067
2019-07-17 00:00:00 S2B_MSIL2A_20190717T142049_N0213_R010_T20MQT_20190717T193145 2.131074
2019-07-17 00:00:00 S2B_MSIL2A_20190717T142049_N0213_R010_T20MRS_20190717T193145 2.200809
2019-07-17 00:00:00 S2B_MSIL2A_20190717T142049_N0213_R010_T20MRT_20190717T193145 0.461807
2019-07-17 00:00:00 S2B_MSIL2A_20190717T142049_N0213_R010_T21MTM_20190717T193145 0.595476
2019-07-17 00:00:00 S2B_MSIL2A_20190717T142049_N0213_R010_T21MTN_20190717T193145 0.506153
2019-07-22 00:00:00 S2A_MSIL2A_20190722T142041_N0213_R010_T20MQS_20190722T164016 0.299776
2019-07-22 00:00:00 S2A_MSIL2A_20190722T142041_N0213_R010_T20MQT_20190722T164016 0.125833
2019-07-22 00:00:00 S2A_MSIL2A_20190722T142041_N0213_R010_T20MRS_20190722T164016 0.02207
2019-07-22 00:00:00 S2A_MSIL2A_20190722T142041_N0213_R010_T20MRT_20190722T164016 0.167836
2019-07-22 00:00:00 S2A_MSIL2A_20190722T142041_N0213_R010_T21MTM_20190722T164016 0.001543
2019-07-22

In [29]:
for date_i in cfg['opt_images_dates']:
    opt_d0 = date_i[0]
    opt_d1 = date_i[1]
    opt_collection_i = opt_collection.filterDate(opt_d0, opt_d1)
    opt_mosaic_i = opt_collection_i.mosaic()
    export_opt(opt_mosaic_i, roi, f'{opt_d0}-{opt_d1}')
    export_maskcloud(opt_mosaic_i, roi, f'cloud_{opt_d0}-{opt_d1}')
    map_view.addLayer(opt_mosaic_i.clip(roi), opt_vizParams, f'opt-{opt_d0}-{opt_d1}')
    print({opt_d0}-{opt_d1})
    print(f'{len(opt_collection_i.getInfo()["features"])} images')


    

{'2019-08-06'}
6 images
{'2019-11-24'}
6 images
{'2020-02-17'}
6 images
{'2020-05-02'}
6 images
{'2020-07-31'}
6 images


# Optical Data (Level 1C)

In [13]:
opt_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(start_date, end_date)
opt_collection = opt_collection.filterBounds(roi)
opt_info = opt_collection.getInfo()

In [97]:
for info in opt_info['features']:
    img_date = datetime.datetime.strptime(info['properties']['DATATAKE_IDENTIFIER'][5:13], '%Y%m%d')
    img_lct = info['properties']['PRODUCT_ID']
    cloud = info['properties']['CLOUDY_PIXEL_PERCENTAGE']
    print(img_date, img_lct, cloud)
    #break

2019-06-26 00:00:00 S2A_MSIL1C_20190626T140101_N0207_R067_T21MXM_20190626T154048 0.1723
2019-06-26 00:00:00 S2A_MSIL1C_20190626T140101_N0207_R067_T21MXN_20190626T154048 2.4004
2019-06-26 00:00:00 S2A_MSIL1C_20190626T140101_N0207_R067_T21MYM_20190626T154048 0
2019-06-26 00:00:00 S2A_MSIL1C_20190626T140101_N0207_R067_T21MYN_20190626T154048 0
2019-07-01 00:00:00 S2B_MSIL1C_20190701T140059_N0207_R067_T21MXM_20190701T154033 0
2019-07-01 00:00:00 S2B_MSIL1C_20190701T140059_N0207_R067_T21MXN_20190701T154033 0.0128
2019-07-01 00:00:00 S2B_MSIL1C_20190701T140059_N0207_R067_T21MYM_20190701T154033 0
2019-07-01 00:00:00 S2B_MSIL1C_20190701T140059_N0207_R067_T21MYN_20190701T154033 0
2019-07-06 00:00:00 S2A_MSIL1C_20190706T140101_N0207_R067_T21MXM_20190706T153903 45.7992
2019-07-06 00:00:00 S2A_MSIL1C_20190706T140101_N0207_R067_T21MXN_20190706T153903 70.2141
2019-07-06 00:00:00 S2A_MSIL1C_20190706T140101_N0207_R067_T21MYM_20190706T153903 14.0007
2019-07-06 00:00:00 S2A_MSIL1C_20190706T140101_N0207_R

In [59]:
for date_i in cfg['opt_images_dates']:
    opt_d0 = date_i[0]
    opt_d1 = date_i[1]
    opt_collection_i = opt_collection.filterDate(opt_d0, opt_d1)
    opt_mosaic_i = opt_collection_i.mosaic()
    #export_opt_l1(opt_mosaic_i, roi, f'{opt_d0}-{opt_d1}')
    map_view.addLayer(opt_mosaic_i.clip(roi), opt_vizParams, f'opt-{opt_d0}-{opt_d1}')
    print(f'{opt_d0}-{opt_d1}.tif')
    #print(f'{len(opt_collection_i.getInfo()["features"])} images')    

2019-07-16-2019-07-17.tif
2019-07-31-2019-08-01.tif
2019-08-05-2019-08-06.tif
2020-07-30-2020-07-31.tif
2020-08-19-2020-08-20.tif
2020-06-30-2020-07-01.tif
2021-07-20-2021-07-21.tif
2021-07-30-2021-07-31.tif
2021-08-14-2021-08-15.tif
